<a href="https://colab.research.google.com/github/hhumayune/Fake-News-Detector/blob/master/fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re

# importing datasets
true_path = '/content/drive/MyDrive/News-_dataset/True.csv'
fake_path = '/content/drive/MyDrive/News-_dataset/Fake.csv'

true_df = pd.read_csv(true_path)
fake_df = pd.read_csv(fake_path)

In [ ]:
# briefly visualizing the datasets
display(true_df)
display(fake_df)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [ ]:
# Data preprocessing part
# Data combination
# Adding Labels to both df
true_df['true'] = 1
fake_df['true'] = 0
# Concat. this basically adds true_df and fake_df
df = pd.concat([true_df, fake_df])
# Shuffling the data
from sklearn.utils import shuffle
df = shuffle(df).reset_index(drop=True)
# displaying again
display(df)


,title,text,subject,date,true
0,Trump to Pennsylvania: See you in four years,"HERSHEY, Pa. (Reuters) - Ready for another Don...",politicsNews,"December 16, 2016",1
1,Belgian trial of Paris attacker Abdeslam postp...,BRUSSELS (Reuters) - The Belgian trial of one ...,worldnews,"December 15, 2017",1
2,DNC Chair Not Holding Back On GOP: ‘They Don’...,Democratic National Committee Chairman Tom Per...,News,"April 1, 2017",0
3,"Exclusive: 10,000 UK finance jobs affected in ...","LONDON (Reuters) - Around 10,000 finance jobs ...",worldnews,"September 18, 2017",1
4,Senate confirms top EPA enforcement official,WASHINGTON (Reuters) - The Senate on Thursday ...,politicsNews,"December 7, 2017",1
...,...,...,...,...,...
44893,"North Korea threatens to 'sink' Japan, reduce ...",SEOUL/TOKYO (Reuters) - A North Korean state a...,worldnews,"September 14, 2017",1
44894,BREAKING: GOP Hands Speaker Gavel To Open Bord...,"As a side note, Paul Ryan still needs 18 addit...",politics,"Oct 28, 2015",0
44895,BREAKING NEWS: Leftist Media Publishes Major F...,How many times in one week can ABC News publis...,politics,"Dec 5, 2017",0
44896,Turkish court orders detention of two over U.S...,ISTANBUL (Reuters) - A Turkish court has issue...,worldnews,"November 28, 2017",1


In [ ]:
# Preprocessing functions for eliminating noise
def preprocess_text(text):
    # Replace characters that are not between a to z or A to Z with whitespace
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Convert all characters into lower-case ones. This doesnt really matter if you use BERT
    text = text.lower()
    # Remove inflectional morphemes like "ed", "est", "s", and "ing" from their token stem
    text = re.sub('(ed|est|s|ing)$', '', text)
    return text

# Apply preprocessing to the title column
df['title'] = df['title'].apply(preprocess_text)
display(df)

,title,text,subject,date,true
0,trump to pennsylvania see you in four year,"HERSHEY, Pa. (Reuters) - Ready for another Don...",politicsNews,"December 16, 2016",1
1,belgian trial of paris attacker abdeslam postp...,BRUSSELS (Reuters) - The Belgian trial of one ...,worldnews,"December 15, 2017",1
2,dnc chair not holding back on gop they don ...,Democratic National Committee Chairman Tom Per...,News,"April 1, 2017",0
3,exclusive uk finance jobs affected in ...,"LONDON (Reuters) - Around 10,000 finance jobs ...",worldnews,"September 18, 2017",1
4,senate confirms top epa enforcement official,WASHINGTON (Reuters) - The Senate on Thursday ...,politicsNews,"December 7, 2017",1
...,...,...,...,...,...
44893,north korea threatens to sink japan reduce ...,SEOUL/TOKYO (Reuters) - A North Korean state a...,worldnews,"September 14, 2017",1
44894,breaking gop hands speaker gavel to open bord...,"As a side note, Paul Ryan still needs 18 addit...",politics,"Oct 28, 2015",0
44895,breaking news leftist media publishes major f...,How many times in one week can ABC News publis...,politics,"Dec 5, 2017",0
44896,turkish court orders detention of two over u s...,ISTANBUL (Reuters) - A Turkish court has issue...,worldnews,"November 28, 2017",1


In [ ]:
#  Splitting the Dataset into Training, Validation, Test 

train_val_df = df.sample(frac = 0.8)
test_df = df.drop(train_val_df.index)

train_df = train_val_df.sample(frac = 0.8)
val_df = train_val_df.drop(train_df.index)

# Reset Index
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print('trainset size:', train_df.shape)
print('valset size:', val_df.shape)
print('testset size:', test_df.shape)
     

trainset size: (28734, 5)
valset size: (7184, 5)
testset size: (8980, 5)


In [ ]:
#for IMBALANCED DATA NOT NEEDED
# #shrinking data into 1/10th of the original file
# train_data = 1/10
# val_data = 1/10
# test_data= 1/10

# train_true = train_df[train_df['true'] == 1]
# train_fake = train_df[train_df['true'] == 0]
# train_fake = train_fake.iloc[:round(len(train_fake)*train_data),:]
# train_true_len = len(train_true)
# train_fake_len = len(train_fake)
# train_df = pd.concat([train_true, train_fake])
# train_df = shuffle(train_df).reset_index(drop=True)

# val_true = val_df[val_df['true'] == 1]
# val_fake = val_df[val_df['true'] == 0]
# val_fake = val_fake.iloc[:round(len(val_fake)*val_data),:]
# val_true_len = len(val_true)
# val_fake_len = len(val_fake)
# val_df = pd.concat([val_true, val_fake])
# val_df = shuffle(val_df).reset_index(drop=True)
# test_true = test_df[test_df['true'] == 1]
# test_fake = test_df[test_df['true'] == 0]
# test_fake = test_fake.iloc[:round(len(test_fake)*test_data),:]
# test_true_len = len(test_true)
# test_fake_len = len(test_fake)
# test_df = pd.concat([test_true, test_fake])
# test_df = shuffle(test_df).reset_index(drop=True)

# len_df = pd.DataFrame({'ind': ['Train', 'Validation', 'Test'], 'True': [train_true_len, val_true_len, test_true_len], 'Fake': [train_fake_len, val_fake_len, test_fake_len]})
# len_df.set_index('ind', inplace=True)
# len_df.index.name = None

# len_df

In [ ]:
# dataset to csv
train_df.to_csv('train.tsv', sep='\t', index=False)
val_df.to_csv('val.tsv', sep='\t', index=False)
test_df.to_csv('test.tsv', sep='\t', index=False)

# concat again
df = pd.concat([train_df, val_df, test_df])
display(df)

,title,text,subject,date,true
0,factbox likely players in potential post coup...,HARARE (Reuters) - Zimbabwe s army appears to ...,worldnews,"November 16, 2017",1
1,whoa soros linked group funding ohio s john k...,Soros is a Hungarian immigrant who s doing his...,politics,"Mar 14, 2016",0
2,senate banking chair wants financial rules ref...,(Reuters) - The chairman of the U.S. Senate Ba...,politicsNews,"March 30, 2017",1
3,conservatives desperately turn to social medi...,"Early Sunday morning, Omar Mateen walked into ...",News,"June 12, 2016",0
4,still bitter bill clinton trump knows how to ...,HERE S CLINTON FROM THE LOCAL NEWSPAPER:Former...,politics,"Dec 19, 2016",0
...,...,...,...,...,...
8975,wind energy firms cheer u s senate tax bill f...,(Reuters) - The U.S wind energy industry on Fr...,politicsNews,"November 10, 2017",1
8976,turkey says hopes u s will lift decision on v...,ANKARA (Reuters) - Turkey hopes the United Sta...,worldnews,"October 11, 2017",1
8977,someone posted a photo of steve bannon s gian...,"You know, for being a member of the master ra...",News,"February 24, 2017",0
8978,breaking news leftist media publishes major f...,How many times in one week can ABC News publis...,politics,"Dec 5, 2017",0


In [ ]:
# Word Embedding
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer

# Creating A Tokenizer To Tokenize The Words And Create Sequences Of Tokenized Words
tokenizer = Tokenizer(num_words = total_words)
tokenizer.fit_on_texts(train_df['title'])

train_sequences = tokenizer.texts_to_sequences(train_df['title'])
val_sequences = tokenizer.texts_to_sequences(val_df['title'])
test_sequences = tokenizer.texts_to_sequences(test_df['title'])
     
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Adding Padding
padded_train = pad_sequences(train_sequences,maxlen = 42, padding = 'post', truncating = 'post')
padded_val = pad_sequences(val_sequences,maxlen = 42, padding = 'post', truncating = 'post')
padded_test = pad_sequences(test_sequences,maxlen = 42, padding = 'post', truncating = 'post')

# display(df)
     

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            520       
                                                                 
 dropout_2 (Dropout)         (None, 20, 40)            0         
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout_3 (Dropout)         (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 113,521
Trainable params: 113,521
Non-trainable params: 0
________________________________________________

Epoch 1/3


ValueError: ignored